Use Python to fetch country debt profiles. Use a horizontal bar chart, the y-axis is the list of all countries in the world (for example, Japan, Korea, Philippines, China, and more), and the x-axis is the total debt in $.  Make a ranking label on the outside and the right part of the bar, followed by the number of debt$ label

In [ ]:
import requests
import pandas as pd
import matplotlib.pyplot as plt

# -----------------------------
# Part 1 — Get all countries
# -----------------------------
countries_resp = requests.get(
    "https://api.worldbank.org/v2/country?format=json&per_page=500"
)
all_countries_data = countries_resp.json()[1]
all_countries = pd.DataFrame({
    "country_id": [c['id'] for c in all_countries_data],
    "country_name": [c['name'] for c in all_countries_data]
})

# -----------------------------
# Part 2 — Fetch Debt Data
# -----------------------------
def fetch_debt(indicator, year="2023"):
    url = f"https://api.worldbank.org/v2/country/all/indicator/{indicator}"
    params = {"format": "json", "date": year, "per_page": 5000}
    resp = requests.get(url, params=params).json()
    
    if len(resp) < 2:
        return pd.DataFrame(columns=["country_id", "value"])
    
    records = resp[1]
    data = []
    for r in records:
        data.append({
            "country_id": r["country"]["id"],
            "value": float(r["value"]) if r["value"] is not None else 0
        })
    return pd.DataFrame(data)

# Indicators:
# External Debt (PPG + Private)
external_debt = fetch_debt("DT.DOD.DECT.CD")
# Domestic Government Debt (Local currency converted to USD may be approximated)
domestic_debt = fetch_debt("GC.DOD.TOTL.CN")  # nominal domestic debt

# -----------------------------
# Part 3 — Merge Data
# -----------------------------
df = all_countries.merge(external_debt, how="left", left_on="country_id", right_on="country_id")
df = df.rename(columns={"value": "external_debt_usd"})

df = df.merge(domestic_debt, how="left", left_on="country_id", right_on="country_id")
df = df.rename(columns={"value": "domestic_debt_local"})

# Replace NaN with 0
df[["external_debt_usd", "domestic_debt_local"]] = df[["external_debt_usd", "domestic_debt_local"]].fillna(0)

# -----------------------------
# Part 4 — Approximate Total Debt
# -----------------------------
# For simplicity, assume domestic debt is roughly equivalent to USD (or convert using FX if available)
df["total_debt_usd"] = df["external_debt_usd"] + df["domestic_debt_local"]

# Sort by total debt
df = df.sort_values("total_debt_usd", ascending=False).reset_index(drop=True)

# -----------------------------
# Part 5 — Plot Horizontal Bar Chart
# -----------------------------
MAX_COUNTRIES = 40
plot_df = df.head(MAX_COUNTRIES)

plt.figure(figsize=(12, 12))
bars = plt.barh(plot_df["country_name"], plot_df["total_debt_usd"], color="steelblue")
plt.gca().invert_yaxis()

plt.title("Top Country Total National Debt (Approximate USD)")
plt.xlabel("Debt in USD (Current)")

for i, (country, val) in enumerate(zip(plot_df["country_name"], plot_df["total_debt_usd"]), 1):
    label = f"{i}. {country} – ${val:,.0f}"
    bars[i-1].set_label(label)
    plt.text(val + max(plot_df["total_debt_usd"]) * 0.01,
             bars[i-1].get_y() + bars[i-1].get_height()/2,
             label, va="center", fontsize=9)

plt.tight_layout()
plt.show()
